Downloading images from OSM
---

Thanks to osmnx

In [2]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox

In [3]:
# configure the inline image display
img_folder = 'images'
extension = 'png'
size = 254
dpi = 72

In [ ]:
df = pd.read_csv('./db.csv')

In [27]:
# gdf = gpd.GeoDataFrame(df, geometry = 'wkb_geometry') ; 
df.head()

,ogc_fid,city,st_astext
0,1,Amsterdam,MULTIPOINT Z (4.922454 52.3778 0)
1,2,Amsterdam,MULTIPOINT Z (4.941598 52.373272 0)
2,3,Amsterdam,MULTIPOINT Z (4.873884 52.386222 0)
3,4,Amsterdam,MULTIPOINT Z (4.934852 52.374276 0)
4,5,Amsterdam,MULTIPOINT Z (4.914172 52.377852 0)


In [6]:
from shapely import wkt
def getCoords(df, x) : return tuple(np.array(wkt.loads(df.iloc[x]['st_astext'])[0].xy).squeeze())

def getLonLat(df, idx):
    if 'MULTI' in df.iloc[idx]['st_astext']:
        wktcoords = wkt.loads(df.iloc[idx]['st_astext'])[0]
    else: 
        wktcoords = wkt.loads(df.iloc[idx]['st_astext'])
    return wktcoords.x, wktcoords.y

In [7]:
gb = df.groupby('city')

In [11]:
outF = open("./labels__new.csv", "w")
outF.write('label,lon,lat')
outF.write("\n")

for group, data in gb:
    gf = pd.DataFrame(gb.get_group(group))
    for idx in range(len(gf)):
        place = group
        point = getCoords(gf, idx)[::-1]
        ogc = gf.iloc[idx]['ogc_fid']
        try: 
            label = group + '_' + str(idx+1) + '_' + str(ogc)
            fig, ax = ox.plot_figure_ground(point=point, 
                                            bgcolor='#000000',
                                            network_type='all',
                                            default_width=3,
                                            dpi=dpi,
                                            filename='./{}.png'.format(label),
                                            show=False)
            outF.write(str(label) + ',' + str(point[0]) + ',' + str(point[1]))
            outF.write("\n")
        except Exception as e:
            print(e)
            pass
outF.close()

If you forgot labels... well, get'em back!

In [48]:
import os
import re

In [69]:
pat = r'^(\w\w)_(\d)'

In [80]:
img_labels = []
country_set =[]
for file in os.listdir('./_project/images/'):
    match = re.search(pat, str(file))
    if match:
        if match.group(1) == match.group(1).upper():
            img_labels.append(file.replace('.png', ''))
            country_set.append(match.group(1))

In [81]:
len(img_labels)

2484

In [83]:
list(set(country_set))

['BE',
 'IN',
 'SV',
 'VN',
 'NL',
 'FR',
 'GB',
 'CL',
 'BR',
 'IT',
 'DE',
 'ES',
 'MX',
 'RO',
 'JP']

Now copy over only the labels that exist in the image list

In [85]:
with open("./labels_2.csv", "r") as f:
    lines = f.readlines()

In [87]:
with open("./labels_copy.csv", 'a') as f:
    for line in lines:
        if line.split(',')[0] in img_labels:
            f.write(line)
f.close()